In [1]:
import os

In [2]:
%pwd

'e:\\MLProject\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\MLProject\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    


In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import create_directories, read_yaml

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )

            logger.info(f"{filename} downloaded with following info : \n{header}")

        else:
            logger.info(f"Filename already exist of size:{get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [14]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2026-02-02 18:47:27,502: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-02 18:47:27,504: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-02 18:47:27,505: INFO: common: created directory at: artifacts]
[2026-02-02 18:47:27,505: INFO: common: created directory at: artifacts/data_ingestion]
[2026-02-02 18:47:28,079: INFO: 1185689452: artifacts/data_ingestion/data.zip downloaded with following info : 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9924:0A05:F9CAC:1FBE63:6980A3AB
Accept-Ranges: bytes
Date: Mon, 02 Feb 2026 13:17:28 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210073-BOM
X-Ca